### Testeo de la funcion de setear condiciones de borde al DMPLEX

En primer lugar definimos un dominio de 2 x 2 elementos y ngl=2, o sea, 9 nodos en total.

De esos 9 nodos, 8 se encuentran en el borde. Siguiendo la numeracion de PETSc el nodo 4 ( empezando a contar desde 0) es el que se encuentra en el centro (no tiene condicion de borde).

En esta primer celda definimos los **imports** y una funcion para asignar los grados de libertad de acuerdo a si es cara, edge o vertice.

In [57]:
from petsc4py import PETSc
import numpy as np

def getNumCompAndNumDof(ngl, componentsPerField ,numFields):
    numComp = [componentsPerField] * numFields
    nodesPerEntity = [ 1 , ngl - 2 , (ngl - 2)**2 ]
    #           vertex-^ edges -^     faces -^   
    numDof = [componentsPerField * nodes for nodes in nodesPerEntity]
    return numComp, numDof

### Creacion del DMPlex

Primero se crea el ```dm``` del tipo box, 2x2, y con ```dm.setNumFields(1)``` se establece que va a tener un unico ```field```, en este caso el field seran los nodos (aunque en un futuro podria directamente tener dos fields, la velocidad y la vorticidad directamente)

Ademas creamos otro ```dmBC``` que va a seguir el mismo proceso pero con condiciones de borde

In [112]:
dm = PETSc.DMPlex().createBoxMesh(faces=[2,2], simplex=False)
dm.setNumFields(1)

dmBC = PETSc.DMPlex().createBoxMesh(faces=[2,2], simplex=False)
dmBC.setNumFields(1)

numOfFields = 1
dofsPerNode = 1
ngl = 2
numComp, numDof = getNumCompAndNumDof(ngl, dofsPerNode , numOfFields)

print("Cantidad de componentes por field", numComp)
print("Grados de libertad por entidad [ vertices , edges , cells]", numDof)

Cantidad de componentes por field [1]
Grados de libertad por entidad [ vertices , edges , cells] [1, 0, 0]


### Seteo del Section y las condiciones de borde

En primer lugar le pedimos al ```dm``` las **entidades** que esten en los bordes.
Esto es algo nuevo que descubri, si le pedimos las entidades con el label "marker" y value=1 nos da las entidades en los bordes

```bcIs``` es un objeto tipo PETSc.IS.

Al llamar a ```dm.createSection()``` los primeros dos parametros son los que ya vimos en el ```print``` de la celda anterior.


```sec``` esta definido tal cual esta hecho ahora
```secBC``` es lo que estamos investigando ahora

Los dos parametros adicionales son para definir las condiciones de borde:
    
El primero es ```bcField``` que en este caso es 0 porque es el unico que hay.

Luego ```bcPoints``` tiene que ser ***necesariamente*** una lista con PETSc.IS. En este caso le pase una con todas las entidades en el borde

Finalmente le damos un nombre a esta ```sec``` y terminamos con ```setUp```

In [113]:
bcIs = dm.getStratumIS("marker", 1)

sec = dm.createSection(numComp, numDof)
secBC = dmBC.createSection(numComp, numDof, 0 , bcPoints=[bcIs])

sec.setFieldName(0, 'Nodes')
secBC.setFieldName(0, 'Nodes')

secBC.setUp()

### Creando la matriz

Para crear la matriz primero debemos setear la ```section``` por default al ```dm``` para luego pedirle la matriz.

En este caso creamos dos matrices, tener en cuenta que estamos haciendo de cuenta que cada nodo tiene 1 grado de libertad, entonces la matriz sin definir condiciones de borde es de 9x9 y definiendo condiciones de borde es de 1x1

In [118]:
dm.setDefaultSection(sec)
mat = dm.createMat()

dmBC.setDefaultSection(secBC)
matBC = dmBC.createMat()

print("Dimension de matriz sin BC" ,mat.getSize())
print("Dimension de matriz con BC",matBC.getSize())

Dimension de matriz sin BC (9, 9)
Dimension de matriz con BC (1, 1)


### Estudiando algunos metodos

Sobre el ```dmBC``` le pedimos los siguientes los vectores denominados locales y globales.

In [120]:
localVec = dmBC.getLocalVec()
globalVec = dmBC.getGlobalVec()

print("Vec local", localVec.getArray())
print("Vec global", globalVec.getArray())

Vec local [0. 0. 0. 0. 0. 0. 0. 0. 0.]
Vec global [0.]


### Sobre el LGMap

el lgmap (map local global) es una clase de PETSc que tiene el siguiente comportamiento:

En el caso del ```dm``` sin bc es una lista comun ```[0, ...., 8]``` que indica todos los nodos.

Pero para ```dmBC``` la lista tiene la particularidad de tener solos 2 valores, el 0 y el 1, ``` [0, 0, 0, 0, 0, 1, 1, 1, 1]``` . Justo cambia de valo 

In [122]:
lg = dm.getLGMap()

print("LGMap dm sin bc", lg.getIndices())

lgBC = dmBC.getLGMap()

print("LGMap dm con bc", lgBC.getIndices())

LGMap dm sin bc [0 1 2 3 4 5 6 7 8]
LGMap dm con bc [0 0 0 0 0 1 1 1 1]
